Ceneo Scraper

## Struktura opini w serwisie Ceneo.pl

|składowa|selektor|zminenna|
|--------|--------|--------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|span.user-post__author-name|author|
|rekomendację|span.user-post__author-recomendation > em|recommend|
|liczbę gwiazdek|span.user-post__score-count|stars|
|treść opinii|div.user-post__text|content|
|listę wad|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|listę zalet|div.review-feature__title--positives ~ div.review-feature__item|pros|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|


In [ ]:
selectors = {
    "opinion_id": (None,"data-entry-id"),
    "author": ('span.user-post__author-name',),
    "recommend": ('span.user-post__author-recomendation > em',),
    "stars": ('span.user-post__score-count',),
    "content": ('div.user-post__text',),
    "cons": ('div.review-feature__title--negatives ~ div.review-feature__item', None, True),
    "pros": ('div.review-feature__title--positives ~ div.review-feature__item', None, True),
    "opinion_date": ('span.user-post__published > time:nth-child(1)',"datetime"),
    "purchase_date": ('span.user-post__published > time:nth-child(2)',"datetime"),
    "up_vote": ('button.vote-yes',"data-total-vote"),
    "down_vote": ('button.vote-no',"data-total-vote"),
}

Funkcja do ekstrakcji kodu HTML

In [ ]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]


    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

## Import bibliotek

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Wczytywanie kodu produktu o którym mają zostać pobrane opinie

In [ ]:
#product_id = "104109736"
product_id = input('Podaj kod produktu: ')
url = "https://www.ceneo.pl/{product_id}#tab=reviews"


## Wysłanie do serwera Ceneo.pl żądania do strony z opiniami o produkcie

In [41]:
response = requests.get(url)

## Przekształcenie kodu HTML z postaci tekstowej do obiektu reprezentująceg DOM
## Wydobycie ze struktury DOM znaczników odpowiadających opini konsumentów

## Wydobycie z struktury znacznika odpowiadającego pojedyńczej opini jej składowych

Zapis opinii o proodukcie do pliku JSON do katalogu opinions

In [ ]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open('opinions/{product_id}.json', "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)


[{'opinion_id': '16872518',
  'author': 'm...7',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Świetny przenośny, bezprzewodowy głośnik. Wykonany z bardzo dobrych materiałów. Dźwięk czysty, wyrazisty i bardzo miły dla ucha. Super basy sprawiają jeszcze większą przyjemność z odsłuchu.',
  'cons': [],
  'pros': [],
  'opinion_date': '2022-12-13 13:48:41',
  'purchase_date': '2022-12-01 07:00:54',
  'up_vote': '5',
  'down_vote': '3'},
 {'opinion_id': '14593429',
  'author': 'Poznawszy',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Polecam',
  'cons': [],
  'pros': [],
  'opinion_date': '2021-06-12 22:21:42',
  'purchase_date': '2021-05-13 12:45:06',
  'up_vote': '9',
  'down_vote': '5'},
 {'opinion_id': '16409183',
  'author': 'k...5',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Głośnik naprawdę super, jakoś dźwięku idzie w parze wraz ze zwiększaniem głośności, bas jest przyjemny dla ucha, na imprezy w plenerze jest to najlepszy wybór ze względu na 

Pobranie wszystkich opinii o wskazanym produkcie z serwisu Ceneo.pl

In [5]:
all_opinions = []
while(url):
    response = requests.get(url) 
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page,'a.pagination__next', "href")
    except TypeError:
        url = None

NameError: name 'url' is not defined